<a href="https://colab.research.google.com/github/kiryu-3/Prmn2023/blob/main/Python/Python_Machine/Machine_Learning_2_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ダミー変数化と係数の仮説検定ー知識編

In [1]:
# 最初にインポートしてください
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import Ridge, LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.model_selection import KFold, RepeatedKFold, cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.datasets import fetch_openml
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.neighbors import KNeighborsRegressor
from sklearn.pipeline import Pipeline
import statsmodels.api as sma

## 質的変数と量的変数

数値で表せる変数のことを、**連続変数**や**量的変数**といったりします。

対して、カテゴリーで表す変数のことを、**カテゴリー変数**や**質的変数**といったりします。

**質的変数は数値ではないので、そのまま機械学習のモデルに特徴量として使うことができません。**

機械学習をする際には質的変数があるデータを扱うことがほとんどです。

ある工夫をすることで、うまく数値で扱えるようになります。

## one-hotエンコーディング

カテゴリーのとりうる値の数だけを0と1で構成するベクトルを  
**one-hot ベクトル(one-hot encoded vector)**といいます。

このベクトルを使って、そのカテゴリーだけを1(hot)で表す処理を行います（**one-hot エンコード**）。


one-hot エンコードは、質的変数を説明変数として使う際に行う変換の中でも  
最も一般的なやり方になります。

以下の図のようなイメージです。

![](https://imgur.com/gOMewKl.png)

one-hot エンコードには少し罠があります。  

上の図を見たとき、以下のような図の方がイメージしやすいように感じます。

![](https://imgur.com/yiMEC8E.png)

このようにすると、特徴量間に完全な相関が生まれてしまいます（**ダミー変数トラップ**）。

詳しいことは[かめさんの記事](https://datawokagaku.com/one_hot_encoding/#i-2)などを参照してもらえればと思いますが、  
one-hot エンコーディングでダミー変数を作る際には、必ず**カテゴリー数-1**の数だけ作るようにします。

## Pythonでone-hotエンコーディング

では、実際にPythonでone-hotエンコーディングを行っていきましょう。

Pythonはたった一行でone-hotエンコーディングを行ってくれます。

### 使用するデータセット

今回は、seabornのサンプル用データセット"tips"を利用します。

データセットは、pandas.DataFrameオブジェクトとして取得することができます。

（参考サイト：[こちら](https://biotech-lab.org/articles/1408#i)）

In [2]:
tips = sns.load_dataset('tips')
tips.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 244 entries, 0 to 243
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   total_bill  244 non-null    float64 
 1   tip         244 non-null    float64 
 2   sex         244 non-null    category
 3   smoker      244 non-null    category
 4   day         244 non-null    category
 5   time        244 non-null    category
 6   size        244 non-null    int64   
dtypes: category(4), float64(2), int64(1)
memory usage: 7.4 KB


"tips"のデータの説明は以下の通りです。

- total_bill : 総支払額(食事代、税込み)　(USドル)
- tip : チップ(USドル)
- sex : 性別
- smoker : 喫煙者か否か
- day : 曜日(木・金・土・日のいずれか)
- time : 食事の時間(昼食か夕食か)
- size : 人数

また、今回のデータには欠損値はなさそうです。

### Pythonによる実行

7つのカラムの内、カテゴリ変数となっているのは4つです。

一つずつ、ユニークな値とその数を確認してみましょう。

In [3]:
print(tips["sex"].value_counts())  # 性別（男or女）
print("------------------------")
print(tips["smoker"].value_counts())  # 喫煙者かどうか（True or False）
print("------------------------")
print(tips["day"].value_counts())  # 曜日（木 or 金 or 土 pr 日）
print("------------------------")
print(tips["time"].value_counts())  # 食事の時間（昼食 or 夕食）

Male      157
Female     87
Name: sex, dtype: int64
------------------------
No     151
Yes     93
Name: smoker, dtype: int64
------------------------
Sat     87
Sun     76
Thur    62
Fri     19
Name: day, dtype: int64
------------------------
Dinner    176
Lunch      68
Name: time, dtype: int64


これらのカラムは数値型のカラムではないため、  
そのまま機械学習のモデルに特徴量として使うことができません。

ここで、one-hotエンコーディングをしてみましょう。

In [4]:
tips = pd.get_dummies(tips, drop_first=True)
tips.head()

,total_bill,tip,size,sex_Female,smoker_No,day_Fri,day_Sat,day_Sun,time_Dinner
0,16.99,1.01,2,1,1,0,0,1,1
1,10.34,1.66,3,0,1,0,0,1,1
2,21.01,3.50,3,0,1,0,0,1,1
3,23.68,3.31,2,0,1,0,0,1,1
4,24.59,3.61,4,1,1,0,0,1,1


それぞれの値がそれぞれの特徴量に変換され、0と1にエンコードされているのが分かります。

引数で`drop_first=True`とすることで、ダミー変数トラップの問題を解消できます。  
（最初のカテゴリの値の特徴量を落とす）

## 線形回帰の係数

このone-hotエンコーディングをしたtipsのデータを使って、  
"total_bill"を予測する重回帰分析を行ってみましょう。

In [5]:
# 目的変数に"total_bill"、説明変数にそれ以外のカラムを指定
x = tips.drop('total_bill', axis=1)
y = tips['total_bill']

# データを訓練データとテストデータに分ける
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=0)

# 説明変数のデータを標準化
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(x_train)
x_train_scaled = scaler.transform(x_train)

# モデルの初期化と学習
model = LinearRegression()
model.fit(x_train_scaled, y_train)

# coefficient  係数
print(pd.Series(model.coef_, index=x.columns))

tip            4.409546
size           3.142303
sex_Female     0.343710
smoker_No     -1.414742
day_Fri       -1.575064
day_Sat       -3.140102
day_Sun       -3.111760
time_Dinner    3.782092
dtype: float64


今回は何も考えずデータをすべて突っ込んだので、  
モデルの精度としては決して高くはありません。

In [6]:
# 学習データの平均と標準偏差を使って標準化する
x_test_scaled = scaler.transform(x_test)

print('訓練データに対する決定係数：', model.score(x_train_scaled, y_train))
print('テストデータに対する決定係数：', model.score(x_test_scaled, y_test))

訓練データに対する決定係数： 0.5602265384805525
テストデータに対する決定係数： 0.5693293373056256


係数の値の大きさは，その特徴量の目的変数に対する影響力の大きさといえます。

"sex_Female"カラムのように、係数がとても小さい場合、  
もしかしたらその特徴量は目的変数とは関係がないかもしれません。

線形回帰の結果の係数を見れば，その特徴量が目的変数に対して影響がないのかどうか分かります。  

これを見るのが**係数の仮説検定**です。

## 係数の仮説検定

以下の帰無仮説と対立仮説の元、係数の仮説検定を行います。

$$
t=\frac{\hat{\theta_j}-0}{SE(\hat{\theta_j})}
$$

- 以下のt値が自由度$m−n−1$($m$:データ数，$n$:特徴量数)のt分布に従う
- $\hat{\theta_j}$：線形回帰から得られた$j$番目の特徴量の係数
- $SE(\hat{\theta_j})$：$\hat{\theta_j}$の標準誤差(standard error)



帰無仮説：$\theta_j=0$  
対立仮説：$\theta_j\neq0$

この結果が有意差があるかを見るには，最終的にp値を計算する必要があります。

有意水準は**5%**とするとすることが多いため、p値が0.05を上回った係数は  
有意差ありとは言い切れないいう結果になります。

このようにして不要な説明変数の数を減らすことができれば、  
モデルの解釈性を高めることができます。

## Pythonで係数の仮説検定

では、実際にPythonで係数の仮説検定を行っていきましょう。

`statsmodels.api` というモジュールにある `OLS` というクラスを使います。

In [ ]:
# 目的変数に"total_bill"、説明変数にそれ以外のカラムを指定
X = tips.drop('total_bill', axis=1)
y = tips['total_bill']

# 標準化して仮説検定
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_scaled = sma.add_constant(X_scaled)
est = sma.OLS(y, X_scaled)
est_trained = est.fit()
print(est_trained.summary())

                            OLS Regression Results                            
Dep. Variable:             total_bill   R-squared:                       0.582
Model:                            OLS   Adj. R-squared:                  0.567
Method:                 Least Squares   F-statistic:                     40.83
Date:                Sun, 17 Sep 2023   Prob (F-statistic):           1.99e-40
Time:                        09:59:06   Log-Likelihood:                -772.89
No. Observations:                 244   AIC:                             1564.
Df Residuals:                     235   BIC:                             1595.
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         19.7859      0.375     52.778      0.0

- "sex_Female"(="x3")
- "day_Fri"(="x5")
- "day_Sat"(="x6")  
- "day_Sun"(="x7")
- "time_Dinner"(="x8")  

以上の5つのカラムは、p値が0.05を上回っているため、   
 ”total_bill”の予測には不要であるという結果が導かれました。